# Testing the spacy Tokenizer

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
# Fix a spacy-Jupyter bug
# https://github.com/jupyter/notebook/issues/4369#issuecomment-459377498
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [4]:
import gensim
import glob
import pandas as pd
import re
import spacy
import sys

sys.path.append('../../')
import tmx

INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


## Loading the Corpus

Our `ArxivCorpus` object lets us iterate through the corpus and apply our preprocessors.

In [5]:
!ls -l ../../data/arxiv/text/

total 56
drwxr-xr-x  3392 tshafer  staff  108544 Feb  6 13:16 0510
drwxr-xr-x  3032 tshafer  staff   97024 Feb  6 13:19 0808
drwxr-xr-x  4596 tshafer  staff  147072 Feb  6 09:16 1208
drwxr-xr-x  5713 tshafer  staff  182816 Feb  6 19:04 1403
drwxr-xr-x   183 tshafer  staff    5856 Feb  6 13:20 9210
drwxr-xr-x   846 tshafer  staff   27072 Feb  6 13:20 9604
-rw-r--r--     1 tshafer  staff   19465 Feb  7 09:11 README.md
-rw-r--r--     1 tshafer  staff    3244 Feb  7 08:17 document_counts.csv
-rw-r--r--     1 tshafer  staff     749 Feb  8 15:24 get-random-data.py


In [6]:
corpus = tmx.corpora.ArxivCorpus(input=glob.glob('../../data/arxiv/text/[0-9]*'), dictionary=[])

INFO:tmx.corpora.arxiv:Input stream provided but dictionary already initialized


In [7]:
next(iter(corpus.getstream()))['text'][:30]

'Steady-state mechanical squeez'

## spacy Tokenizer

Despite *spacy*'s odd missing of plurals in title case, we'll use it because it seems to do well in other places.

 1. Collapse hyphenated words
 2. Run *spacy*
 3. Remove unwanted tokens


In [8]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [9]:
doc = next(iter(corpus.getstream()))['text'] + '\n\nThis is a new non-specialized sentence.'

In [11]:
hyphens = [{'f': r'non[-–]+', 'r': 'non'}, {'f': r'x[-–]ray', 'r': 'xray'}, {'f': r'space[-–]+time', 'r': 'spacetime'}]
for h in hyphens:
    rx = re.compile(h['f'], re.I)
    doc = rx.sub(h['r'], doc)

In [12]:
doc = nlp(gensim.corpora.textcorpus.deaccent(doc))

In [13]:
tokens = [
    {'token': t.text, 'lemma': t.lemma_}
    for t in doc
    if re.search(r'[\w]', t.lemma_) is not None
    and re.search(r'[\[\](){}]', t.lemma_) is None
    and t.lemma_ != '-PRON-'
]

In [14]:
tokens

[{'token': 'Steady', 'lemma': 'steady'},
 {'token': 'state', 'lemma': 'state'},
 {'token': 'mechanical', 'lemma': 'mechanical'},
 {'token': 'squeezing', 'lemma': 'squeezing'},
 {'token': 'in', 'lemma': 'in'},
 {'token': 'an', 'lemma': 'an'},
 {'token': 'optomechanical', 'lemma': 'optomechanical'},
 {'token': 'system', 'lemma': 'system'},
 {'token': 'via', 'lemma': 'via'},
 {'token': 'Duffing', 'lemma': 'duffing'},
 {'token': 'nonlinearity', 'lemma': 'nonlinearity'},
 {'token': 'Xin', 'lemma': 'xin'},
 {'token': 'Lu', 'lemma': 'lu'},
 {'token': 'Jie', 'lemma': 'jie'},
 {'token': 'Qiao', 'lemma': 'qiao'},
 {'token': 'Liao', 'lemma': 'liao'},
 {'token': 'Lin', 'lemma': 'lin'},
 {'token': 'Tian', 'lemma': 'tian'},
 {'token': 'Franco', 'lemma': 'franco'},
 {'token': 'Nori', 'lemma': 'nori'},
 {'token': 'Introduction', 'lemma': 'introduction'},
 {'token': 'Introduction', 'lemma': 'introduction'},
 {'token': 'sec1', 'lemma': 'sec1'},
 {'token': 'Enormous', 'lemma': 'enormous'},
 {'token': 'pr

## Hand-Wired Tokenizer

I've already built a hand-wired tokenizer for this corpus.

In [25]:
def count_in_list(lst, doc):
    counts = {}
    for item in lst:
        if item not in counts:
            counts[item] = 0
        counts[item] += 1
    return [{'doc': doc, 'token': item, 'count': counts[item]} for item in counts]

In [8]:
token_set = []
stem_set = []
for i, doc in enumerate(corpus.getstream()):
    identifier = doc['meta']
    tokens = tmx.corpora.ArxivCorpus.preprocess_text(doc['text'], stop=False, stem=False)
    stems = tmx.corpora.ArxivCorpus.preprocess_text(doc['text'], stop=False, stem=True)
    token_set.extend(count_in_list(tokens, identifier))
    stem_set.extend(count_in_list(stems, identifier))
    if i >= 1000:
        break

In [105]:
tokens = pd.DataFrame(token_set).sort_values('doc').sort_values('count', ascending=False)
stems = pd.DataFrame(stem_set).sort_values('doc').sort_values('count', ascending=False)

In [106]:
token_counts = tokens.groupby('token')['count'].agg('sum').to_frame().reset_index().sort_values('count', ascending=False)
stem_counts = stem.groupby('token')['count'].agg('sum').to_frame().reset_index().sort_values('count', ascending=False)

In [107]:
token_counts = token_counts[~token_counts['token'].isin(gensim.corpora.textcorpus.STOPWORDS)]
stem_counts = stem_counts[~stem_counts['token'].isin(gensim.corpora.textcorpus.STOPWORDS)]

In [108]:
token_counts.head(n=10)

token  count
34820       let  12610
8968       case  10024
30447         j   9995
36257         m   9868
22417  function   9750
23826     given   9018
55949       set   8843
39633     model   8286
53764         s   8065
62587      time   7921

In [110]:
stem_counts.head(n=10)

token  count
377     mechan     80
586     squeez     61
384       mode     52
458        phy     44
408  nonlinear     42
362          m     42
314          j     40
523        rev     38
86      caviti     33
592      state     32

## Spacy's Tokenizer

In [9]:
!python -m spacy download en_core_web_sm

Looking in indexes: http://nexus.cho.elderresearch.com:8081/repository/eri-pypi-all/simple/, https://pypi.python.org/simple

    Linking successful
    /usr/local/miniconda3/envs/Belair-TM/lib/python3.6/site-packages/en_core_web_sm
    -->
    /usr/local/miniconda3/envs/Belair-TM/lib/python3.6/site-packages/spacy/data/en_core_web_sm

    You can now load the model via spacy.load('en_core_web_sm')



### Pipeline Construction

I don't particularly care about NLP features (e.g., named-entity recognition) for this project. What can I disable without affecting the tokenizer?

In [15]:
nlp = spacy.load('en', disable=[])
pipeline_full = nlp(next(iter(corpus.getstream()))['text'])

In [16]:
nlp = spacy.load('en', disable=['ner'])
pipeline_no_ner = nlp(next(iter(corpus.getstream()))['text'])

In [17]:
nlp = spacy.load('en', disable=['tagger'])
pipeline_no_tagger = nlp(next(iter(corpus.getstream()))['text'])

In [18]:
nlp = spacy.load('en', disable=['parser'])
pipeline_no_parser = nlp(next(iter(corpus.getstream()))['text'])

The pipeline does _not_ matter at all for the tokenizer:

In [19]:
print(sum(pipeline_full[i].text != pipeline_no_ner[i].text for i, _ in enumerate(pipeline_full)))
print(sum(pipeline_full[i].text != pipeline_no_tagger[i].text for i, _ in enumerate(pipeline_full)))
print(sum(pipeline_full[i].text != pipeline_no_parser[i].text for i, _ in enumerate(pipeline_full)))

0
0
0


But, the tagger is required to properly lemmatize the tokens:

In [20]:
print(sum(pipeline_full[i].lemma_ != pipeline_no_ner[i].lemma_ for i, _ in enumerate(pipeline_full)))
print(sum(pipeline_full[i].lemma_ != pipeline_no_tagger[i].lemma_ for i, _ in enumerate(pipeline_full)))
print(sum(pipeline_full[i].lemma_ != pipeline_no_parser[i].lemma_ for i, _ in enumerate(pipeline_full)))

0
1340
0


E.g.:

In [21]:
for i, _ in enumerate(pipeline_full):
    full = pipeline_full[i].lemma_
    no_tagger = pipeline_no_tagger[i].lemma_
    if full != no_tagger:
        print(i, full, no_tagger)
    if i > 25:
        break

0 steady Steady
4 squeezing squeeze
6 an a
11 duffing Duffing
14 xin Xin
16 -PRON- You
17 lü Lü
19 jie Jie
21 qiao Qiao
22 liao Liao
24 lin Lin
25 tian Tian


**Conclusion:** We should keep the tagger. We also need to normalize the text.

### Compare Tokenizers on My Sample Corpus

In [22]:
x=next(iter(nlp("heres a sentence")))

In [23]:
x.text

'heres'

In [26]:
token_set = []
stem_set = []

nlp = spacy.load('en', disable=['ner'])
for i, doc in enumerate(corpus.getstream()):
    identifier = doc['meta']
    # deaccent
    processed_doc = nlp(gensim.parsing.preprocessing.utils.deaccent(doc['text']))
    # convert tokens to lower case
    tokens = [t.text for t in processed_doc]
    stems = [t.lemma_ for t in processed_doc]
    token_set.extend(count_in_list(tokens, identifier))
    stem_set.extend(count_in_list(stems, identifier))
    if i >= 10:
        break

spacy requires addition work because it keeps in non-word tokens.

In [27]:
spacy_token_counts = pd.DataFrame(token_set).groupby('token')['count'].agg('sum').to_frame().reset_index().sort_values('count', ascending=False)
spacy_token_counts = spacy_token_counts[~spacy_token_counts['token'].isin(gensim.corpora.textcorpus.STOPWORDS)]
spacy_token_counts = spacy_token_counts[~spacy_token_counts['token'].apply(lambda f: re.search(r'[a-zA-Z]', f) is None)]

In [28]:
spacy_stem_counts = pd.DataFrame(stem_set).groupby('token')['count'].agg('sum').to_frame().reset_index().sort_values('count', ascending=False)
spacy_stem_counts = spacy_stem_counts[~spacy_stem_counts['token'].isin(gensim.corpora.textcorpus.STOPWORDS)]
spacy_stem_counts = spacy_stem_counts[~spacy_stem_counts['token'].apply(lambda f: re.search(r'[a-zA-Z]', f) is None)]

In [29]:
spacy_stem_counts.head()

token  count
72    -PRON-    965
3969    phys    311
5093    time    252
2197      eq    244
3022      j.    220

In [30]:
stem_counts.head()

NameError: name 'stem_counts' is not defined

In [31]:
# Weird plurals to hunt down
spacy_stem_counts[spacy_stem_counts['token'].apply(lambda t: re.search(r'mechan', t) is not None)]

token  count
3447            mechanical     80
3804        optomechanical     16
3449             mechanism      4
3803          optomechanic      1
4552  sec3[sec3]mechanical      1
3624         nanomechanics      1
3446              mechanic      1
3448             mechanics      1

**TODO:** how do we get 'mechanisms'?

In [32]:
spacy_stem_counts[spacy_stem_counts['token'].apply(lambda t: re.search(r'[\[\]{}()]', t) is not None)].head()

token  count
2202  eq13[eq13     24
2200  eq11[eq11     18
2201  eq12[eq12     16
2212    eq2[eq2     12
2214    eq6[eq6     10

This _does_ look better, as if we could use spacy and just drop tokens with braces. **TODO:** suffix, prefix deletions?

### Weird plurals

It's just inconsistent. But pretty close.

In [33]:
nlp.pipeline

[('tagger', <spacy.pipeline.Tagger at 0x1a2a10db00>),
 ('parser', <spacy.pipeline.DependencyParser at 0x1a1e38f830>)]

In [34]:
tmp = pd.DataFrame(stem_set)

In [35]:
tmp[tmp['token'] == 'mechanisms']['doc']

Series([], Name: doc, dtype: object)

In [36]:
for doc in corpus.getstream():
    if doc['meta'] == '1403.2715': #0034':
        print(re.search(r'[^\s]+\s*mechanism\s*[^\s]+',doc['text'],re.I))
        break

<_sre.SRE_Match object; span=(88434, 88454), match='"Physical Mechanisms'>


In [37]:
nlp = spacy.load('en', disable=['ner', 'parser'])
for doc in corpus.getstream():
    if doc['meta'] == '1403.2715':
        parsed = nlp(doc['text'])
        break

In [38]:
nlp = spacy.load('en', disable=['ner', 'parser'])
for doc in corpus.getstream():
    if doc['meta'] == '1403.2715':
        parsed_x = nlp(doc['text'].upper())
        break

In [39]:
[t.lemma_ for t in nlp("I'm governing here!")]

['-PRON-', 'be', 'govern', 'here', '!']

In [40]:
for i, _ in enumerate(parsed):
    if parsed[i].lemma_ != parsed_x[i].lemma_ and parsed_x[i].lemma_.strip():
        print(i, parsed[i], parsed[i].lemma_, parsed_x[i], parsed_x[i].lemma_)

4 annulus annulus ANNULUS annulu
16 dynamics dynamic DYNAMICS dynamics
50 Egbers egbers EGBERS egber
79 techniques technique TECHNIQUES techniques
96 processes process PROCESSES processes
117 are be ARE are
134 graining grain GRAINING graining
154 processes process PROCESSES processes
161 governing govern GOVERNING governing
163 themselves -PRON- THEMSELVES themselves
171 inaccuracies inaccuracy INACCURACIES inaccuracies
176 applied apply APPLIED applied
181 equations equation EQUATIONS equations
191 understood understand UNDERSTOOD understood
193 processes process PROCESSES processes
194 are be ARE are
213 errors error ERRORS errors
216 ones one ONES ones
227 poses pose POSES poses
247 tests test TESTS tests
250 controlled control CONTROLLED controlled
291 it -PRON- IT it
298 governing govern GOVERNING governing
306 processes process PROCESSES processes
309 can can CANNOT cannot
310 not not BE be
311 be be STUDIED studied
312 studied study INDEPENDENTLY independently
313 independently

IndexError: [E040] Attempt to access token at 17817, max length 17817.

### Suffixes and prefixes

We can fix some splittings that we don't want using a custom component. See <https://stackoverflow.com/a/50775597/656740> and <https://spacy.io/usage/linguistic-features#rule-based-matching>.

In [41]:
import spacy.matcher

In [84]:
nlp = spacy.load('en', disable=['ner', 'parser'])

In [85]:
mm = spacy.matcher.Matcher(nlp.vocab)
mm.add('blah', None, [{'LOWER': 'non'}, {'ORTH': r'-–—', 'OP': '+'}, {'IS_PUNCT': False, 'IS_SPACE': False, 'IS_STOP': False, 'OP': '+'}])
def hyphen_collapser(doc):
    ms = mm(doc)
    xx = []
    for match_id, start, end in ms:
        print(match_id, start, end)
        span=doc[start:end]
        xx += [span]
    for ss in xx:
        print(ss, type(ss))
        ss.merge()
    return doc

In [86]:
nlp.add_pipe(hyphen_collapser, name='hyphen_collapser', first=True)

In [87]:
[(t, t.lemma_) for t in nlp('This is non-negotiable or super–terrible or hmm--xx!')]

[(This, 'this'),
 (is, 'be'),
 (non, 'non'),
 (-, '-'),
 (negotiable, 'negotiable'),
 (or, 'or'),
 (super, 'super'),
 (–, '–'),
 (terrible, 'terrible'),
 (or, 'or'),
 (hmm, 'hmm'),
 (--, '--'),
 (xx, 'xx'),
 (!, '!')]

### Bigrams